I couldn't use this OpenAI because i didn't had acess to the API

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(temperature = 0.6)

name = llm("what should be the name of a book written in french about american burgers?")
name

# #Day15
Using Llama-cpp LLM model with langchain

In [ ]:
%pip install llama-cpp-python

Ain't working as well

In [ ]:
from langchain.llms import LlamaCpp
llm = LlamaCpp(
    model_path = "/Users/anantupadhyay/Downloads/LLM model/ggml-model-q4_0.gguf",
    temperature = 0.6
)

name = llm("An american burger restaurant in France")


Trying another model

In [12]:
%pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
sys.path.append("/Users/anantupadhyay/Downloads/API")
import api_key
api_hf = api_key.hf_key



In [9]:
from langchain.llms import HuggingFaceHub
os.environ['HUGGINGFACEHUB_API_TOKEN'] = api_hf
llm = HuggingFaceHub(
    repo_id="meta-llama/Llama-2-7b-chat-hf",
    task="text-generation",
    model_kwargs={"temperature": 0.6},
)
print("llm set")
name = llm("Suggest a name for a tech startup")
name

llm set


BadRequestError:  (Request ID: Zc8Eex9ljkzEhSwOzpqbQ)

Bad request:
Authorization header is correct, but the token seems invalid

Down here, is prompt for chaining the particular statement for multiple different templates, the code is pretty self explainatory. the prompt make the statement generic and the chain connects the prompt with the LLM model

In [10]:
from langchain.prompts import PromptTemplate
prompt_template_name = PromptTemplate(
    input_variables = ['business'],
    template = "A {business} in france"
)
prompt_template_name.format(business="burger shop")


'A burger shop in france'

In [11]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Chinese restaurant")


BadRequestError:  (Request ID: ZDrXFDuaGFUuSH7K-A7T0)

Bad request:
Authorization header is correct, but the token seems invalid

# #Day16
LLM agents


In [13]:
%pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=2b8206109849894b375ed53321ba2a6e6f6c84ea861055e1477fb4a87a62b141
  Stored in directory: /Users/anantupadhyay/Library/Caches/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install numexpr

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain.agents import AgentType,load_tools,initialize_agent
from langchain.llms import HuggingFaceHub

tools = load_tools(["wikipedia","llm-math"],llm= llm)
agent =initialize_agent(
        tools,
        llm,
        agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

agent.run("When was Bill Gates born? What is his age now?")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


BadRequestError:  (Request ID: HG8AqJD_LHoLCuC22Mg2v)

Bad request:
Authorization header is correct, but the token seems invalid

LLM memory: We can explicitly attach the memory with the llm model so that it can remember all the previous conversation/inputs/outputs

In [20]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

chain = LLMChain(llm = llm,prompt = prompt_template_name, memory=memory) 
# obviously we can directly call the attributes in the code, it's just a prctice for understanding code

name = chain.run("A burger shop")
name

BadRequestError:  (Request ID: iR_tzYlCqQzRZDNT9Oh-u)

Bad request:
Authorization header is correct, but the token seems invalid

the problem with memory attribute is that, it's gonna keep increasing the memory buffer and that will increase the costing as well.

In [21]:
#to tackle that issue we use window memory buffer

from langchain.memory import ConversationBufferWindowMemory

#everything is same, it's just this limits the conversational length